In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name() 
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-12-12 09:28:15.409645: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-12-12 09:28:15.409659: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-12 09:28:15.409664: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-12 09:28:15.410235: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-12 09:28:15.410705: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler,SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [ ]:
# #@title Specify CUDA as device for Torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

In [20]:
device = torch.device("mps")
print(device)

mps


In [21]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv("in_domain_train.tsv", delimiter='\t', header=None,names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.shape

(8551, 4)

In [25]:
df

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,NaN,one more pseudo generalization and i 'm giving...
2,gj04,1,NaN,one more pseudo generalization or i 'm giving ...
3,gj04,1,NaN,"the more we study verbs , the crazier they get ."
4,gj04,1,NaN,day by day the facts are getting murkier .
...,...,...,...,...
8546,ad03,0,*,poseidon appears to own a dragon
8547,ad03,0,*,digitize is my happiest memory
8548,ad03,1,NaN,it is easy to slay the gorgon .
8549,ad03,1,NaN,i had the strangest feeling that i knew you .


In [26]:
sentences = df.sentence.values

In [27]:
sentences

array(["our friends wo n't buy this analysis , let alone the next one we propose .",
       "one more pseudo generalization and i 'm giving up .",
       "one more pseudo generalization or i 'm giving up .", ...,
       'it is easy to slay the gorgon .',
       'i had the strangest feeling that i knew you .',
       'what all did you get for christmas ?'], dtype=object)

In [28]:
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences] 
labels = df.label.values

In [32]:
sentences

["[CLS] our friends wo n't buy this analysis , let alone the next one we propose . [SEP]",
 "[CLS] one more pseudo generalization and i 'm giving up . [SEP]",
 "[CLS] one more pseudo generalization or i 'm giving up . [SEP]",
 '[CLS] the more we study verbs , the crazier they get . [SEP]',
 '[CLS] day by day the facts are getting murkier . [SEP]',
 "[CLS] i 'll fix you a drink . [SEP]",
 '[CLS] fred watered the plants flat . [SEP]',
 '[CLS] bill coughed his way out of the restaurant . [SEP]',
 "[CLS] we 're dancing the night away . [SEP]",
 '[CLS] herman hammered the metal flat . [SEP]',
 '[CLS] the critics laughed the play off the stage . [SEP]',
 '[CLS] the pond froze solid . [SEP]',
 '[CLS] bill rolled out of the room . [SEP]',
 '[CLS] the gardener watered the flowers flat . [SEP]',
 '[CLS] the gardener watered the flowers . [SEP]',
 '[CLS] bill broke the bathtub into pieces . [SEP]',
 '[CLS] bill broke the bathtub . [SEP]',
 '[CLS] they drank the pub dry . [SEP]',
 '[CLS] they dran

In [30]:
labels

array([1, 1, 1, ..., 1, 1, 1])